In [1]:

# Importa as bibliotecas necessárias
import warnings
from selenium import webdriver
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from time import sleep
import pandas as pd
import numpy as np

In [2]:
# Inicializa listas para armazenar os dados
data_horario = []
data_home = []
data_away = []
data_odds_home = []
data_odds_draw = []
data_odds_away = []

driver = webdriver.Edge()

pages = 5
for p in range(1, pages + 1):
    driver.get(f'https://www.betfair.com/exchange/plus/pt/futebol-apostas-1/today/{p}')
    driver.implicitly_wait(5)
    driver.maximize_window()

    try:
        # Fechar o pop-up de cookies
        driver.find_element(By.XPATH, "//button[@id='onetrust-accept-btn-handler']").click()
    except:
        pass
    driver.implicitly_wait(5)

    horario = driver.find_elements(By.CLASS_NAME, 'start-date-wrapper')
    clubes = driver.find_elements(By.CLASS_NAME, 'runners')
    odds = driver.find_elements(By.CLASS_NAME, "Zs3u5")

    for i in range(len(horario)):
        home, away = clubes[i].text.split("\n")
        # Adiciona os dados coletados às listas
        data_horario.append(horario[i].text)
        data_home.append(home)
        data_away.append(away)
        data_odds_home.append(odds[(i*6)+0].text)
        data_odds_away.append(odds[(i*6)+4].text)
        data_odds_draw.append(odds[(i*6)+2].text)

driver.close()

# Cria um DataFrame com os dados coletados
df = pd.DataFrame({
    'Horario': data_horario,
    'Time de Casa': data_home,
    'Time Visitante': data_away,
    'Odds Casa': data_odds_home,
    'Odds Visitante': data_odds_away,
    'Odds Empate': data_odds_draw
})

df["JUICE"] = (1/pd.to_numeric(df["Odds Casa"])) + (1/pd.to_numeric(df["Odds Visitante"])) + (1/pd.to_numeric(df["Odds Empate"]))
df = df[df["JUICE"] <= 2]

# Salva o DataFrame em um arquivo CSV
df.to_csv('odds_betfair.csv', index=False, sep=';', encoding='utf-8-sig')